In [5]:
import os
import sys
import pandas as pd

from trading_platform.core.services.file_service import FileService

In [84]:
def group_by_freq(self, input_dir, output_dir, windows_per_file: 60, freq='min'):
    """
    Group ticker files by frequency and write to csv.
    """
    self.file_service.create_dir_if_null(output_dir)
    glob_path = os.path.join(input_dir, '**', '*ticker*.csv')
    ticker_filenames: List[str] = glob.glob(glob_path, recursive=True)

    windows_added: int = 0
    ticker_dfs: List[pandas.DataFrame] = []

    for ticker_filename in ticker_filenames:
        if windows_added < windows_per_file:
            ticker_df = TickerEtlService.read_csv(ticker_filename, True)
            if ticker_df is not None:
                ticker_dfs.append(ticker_df)
            windows_added += 1
        else:
            if len(ticker_dfs) > 0:
                ticker_grouped_df: pandas.DataFrame = pandas.concat(ticker_dfs)
                ticker_grouped_df = ticker_grouped_df.groupby(pandas.Grouper(key='app_create_timestamp', freq=freq))
                earliest_window: str = ticker_grouped_df.index[0]

                ticker_grouped_df.to_csv(os.path.join(output_dir, 'ticker_agg_{0}'.format(earliest_window)))

            windows_added = 0
            ticker_dfs = []

relative_dir: str = 'trading_platform/notebooks'            
# input_dir: str = os.path.dirname(__file__).replace(relative_dir, 'trading_system_data/tickers/raw')
# output_dir: str = os.path.dirname(__file__).replace(relative_dir, 'trading_system_data/tickers/grouped')
# group_by_freq(self, input_dir, output_dir, windows_per_file: 60, freq='min')

def read_csv(filename, parse_dates: bool):
    try:
        if parse_dates:
            def second_parser(arg):
                return pandas.to_datetime(arg=arg, unit='s')

            return pandas.read_csv(filename, date_parser=second_parser, parse_dates=['app_create_timestamp'])
        else:
            return pandas.read_csv(filename)
    # Happens if .csv file is empty
    except EmptyDataError:
        return
df = pd.concat([
    read_csv('/Users/shanekeller/Documents/trading_system_data/tickers/standardized/ticker_v5_2018-07-19T21:16.csv', True),
    read_csv('/Users/shanekeller/Documents/trading_system_data/tickers/standardized/ticker_v5_2018-07-19T21:17.csv', True)
])
df['app_create_timestamp_min']=df['app_create_timestamp'].dt.round('min')
df.set_index('app_create_timestamp_min', inplace=True)

In [102]:
from trading_platform.exchanges.data.ticker import Ticker
from typing import List, Dict

latest_tickers_df = df[df.exchange_id == 1]
tickers: Dict[str, Ticker] = {'{0}_{1}'.format(row['quote'], row['base']): Ticker(**row) for row in
                              latest_tickers_df.to_dict(orient='records')}


tickers['CURE_BTC'].__dict__
# dt_idx = pandas.date_range('20180718', '20180720', freq='min')

# nt = df.loc[name]
# nt[(nt.exchange_id == 1)]
# for idx in dt_idx:
#     if name == idx:
#         for row in df.loc[name].iterrows():

{'ask': 3.0399999999999987e-05,
 'bid': 3.0200000000000012e-05,
 'last': 3.02e-05,
 'base_volume': 12317.42002664,
 'quote_volume': 0.3707227500000002,
 'base': 'BTC',
 'quote': 'CURE',
 'exchange_id': 1,
 'exchange_timestamp': 1532035060055,
 'app_create_timestamp': Timestamp('2018-07-19 21:17:45.659953833'),
 'db_id': None,
 'db_create_timestamp': None,
 'db_update_timestamp': None,
 'version': 5}